# Sentimentanalyse mit Vader II



### Import

In [ ]:
import pandas as pd

### Download mit NLTK
(muss nur einmal gemacht werden)

In [ ]:
import nltk
nltk.download('vader_lexicon')

### Importieren

[NLTK sentiment examples](http://www.nltk.org/howto/sentiment.html)

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

### Sentiment analyzer instanziieren

In [ ]:
sentiment_analyzer = SentimentIntensityAnalyzer()

In [ ]:
sentiment_analyzer.lexicon

In [ ]:
sentiment_analyzer.lexicon["happy"]

In [ ]:
sentiment_analyzer.lexicon["challenging"]

Wir verwenden die Methode `polarity_scores`, die uns die Kategorien
* negativ
* positiv
* neutral
* compound 

zurückliefert. Der `compound score` ist die Summe aus positiven, negativen und neutralen Scores, die dann zwischen -1 (sehr negativ) und +1 (sehr positiv) normalisiert werden. 

In [ ]:
sentiment = sentiment_analyzer.polarity_scores("This is very bad.")
sentiment

### Beispiele

In [ ]:
sentiment = sentiment_analyzer.polarity_scores("This is very bad!")
sentiment

In [ ]:
sentiment = sentiment_analyzer.polarity_scores("This is very bad :( ")
sentiment

In [ ]:
sentiment = sentiment_analyzer.polarity_scores("Profoundly deep, genuinely moving, utterly hilarious, highly imaginative and a visual feast.")
sentiment

### Kommentare laden
Wir laden ein gepickeltes pandas DataFrame. Dabei handelt es sich um ein binäres Dateiformat, das nicht mit einem einfachen Texteditor gelesen werden kann. Die Dateigröße wird jedoch durch die Komprimierung verringert, und im Vergleich zu .csv-Dateien ist es besser für Text geeignet. Reddit ist ein Social-News-Aggregator, eine Website, auf der registrierte Benutzer:innen Inhalte einstellen bzw. anbieten können.

In [ ]:
df = pd.read_csv('archive/data.csv', delimiter=',', header=None, encoding='latin-1')

In [ ]:


df = pd.read_csv('archive/data.csv', delimiter=',')
df.head(10)



In [ ]:
df['Sentiment'].value_counts()

In [ ]:
def sentiment_scores(dataframe):
    sid_obj = SentimentIntensityAnalyzer()
    
    dataframe['compound'] = dataframe['Sentence'].apply(lambda x: sid_obj.polarity_scores(x)['compound'])
    
    def get_sentiment(compound):
        if compound >= 0.05:
            return "positive"
        elif compound <= -0.05:
            return "negative"
        else:
            return "neutral"
        
    dataframe['vader_sentiment'] = dataframe['compound'].apply(get_sentiment)
    return dataframe

df = sentiment_scores(df)

df.head(10)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
accuracy = accuracy_score(df['Sentiment'], df['vader_sentiment'])
precision = precision_score(df['Sentiment'], df['vader_sentiment'], average='macro')
recall = recall_score(df['Sentiment'], df['vader_sentiment'], average='macro')
f1 = f1_score(df['Sentiment'], df['vader_sentiment'], average='macro')

print(f"Vader accuracy: {accuracy}")
print(f"vader precision: {precision}")
print(f"Vader recall: {recall}")
print(f"Vader F1 Score: {f1}")

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming df is your DataFrame

# Create a long-format DataFrame for easier plotting with seaborn
long_format = pd.concat([
    df['Sentiment'].value_counts().rename('count').reset_index().assign(column='Sentiment'),
    df['vader_sentiment'].value_counts().rename('count').reset_index().assign(column='Vader Sentiment')
])

# Plotting
plt.figure(figsize=(8, 6))
sns.barplot(x='index', y='count', hue='column', data=long_format)
plt.xlabel('Kategorie')
plt.ylabel('Gesamtzählung')
plt.title('Vergleich zwischen den originalen Sentiments und den Vader Sentiments')
plt.show()
